Well this idea went pretty much nowhere

The model simply learned to generate a black image, as it was the closest way of getting the right results
It explains why all the accuracies tested generated approx the same avg.

In [ ]:
import os
from lib.train_AI_lib import *
from lib.SemiSup import *
from lib.AutoEncoders import *
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
os.chdir('..')
print(os.getcwd())

## Using auto encoder to semi-supervise

'''
batchsize=16; epoch=15; lr=0.01
enc = autEncE('autEncE'); enc.cuda()
enc.load_state_dict(torch.load('saved/TrainingRuns/AutoEncTrainer/autEncE_b64_te15_lr0.001/model_epoch14'))

net = semiSupB('semiSupB', enc); net.cuda()
trainLoader, valLoader, testLoader = loadData(batchsize=batchsize, preCalc=0)
_ = trainNet(net, [trainLoader, valLoader], batchsize, epoch, lr)
'''

Absolutely Terrible performance, either gets stuck at ~21 rsme or dosen't converge at all. I assume the reason is because the aut. enc is not symm, so the
convolution in the later states (and the data that it extracts based on the code) is not trivial or easy to replicate

Attempting to use the auto encoder generatively

## Using auto encoder generatively

In [ ]:
batchsize=16; epoch=15; lr=0.005
enc = autEncC('autEncC'); enc.cuda()
enc.load_state_dict(torch.load('saved/TrainingRuns/AutoEncTrainer/autEncC_b64_te15_lr0.001/model_epoch14'))

net = genSemiA('genSemiA_encC', enc); net.cuda()
trainLoader, valLoader, testLoader = loadData(batchsize=batchsize, preCalc=0)
_ = trainNet(net, [trainLoader, valLoader], batchsize, epoch, lr, trainType='RegAdam')

Lowest Training   ~15 rmse
Lowest Validation ~16 rmse

In [ ]:
batchsize=16; epoch=15; lr=0.005
enc = autEncD('autEncD'); enc.cuda()
enc.load_state_dict(torch.load('saved/TrainingRuns/AutoEncTrainer/autEncD_b64_te15_lr0.001/model_epoch14'))

net = genSemiA('genSemiA_encD', enc); net.cuda()
trainLoader, valLoader, testLoader = loadData(batchsize=batchsize, preCalc=0)
_ = trainNet(net, [trainLoader, valLoader], batchsize, epoch, lr, trainType='RegAdam')

Lowest Training   ~11 rmse
Lowest Validation ~15 rmse

Overfitting Observed

All three models except encoder E showed promise, All with finial training errors below the baseline. Am begining to wonder, if the lables we have are 'categorized' into the discrete numbers of bounding boxes,
can we round the output of the network then use the rounded output as the input into the loss function? Could the loss function then stay MSE, or could it work as Cross Entropy (most likely a classification function without functions like softmax, but still)

### Attempting to see if i can make this into a sudo-classification problem as defined above

batchsize=16; epoch=15; lr=0.005
enc = autEncD('autEncD'); enc.cuda()
enc.load_state_dict(torch.load('saved/TrainingRuns/AutoEncTrainer/autEncD_b64_te15_lr0.001/model_epoch14'))

net = genSemiA('genDiscA_encD', enc); net.cuda()
trainLoader, valLoader, testLoader = loadData(batchsize=batchsize, preCalc=0)
_ = trainNet(net, [trainLoader, valLoader], batchsize, epoch, lr, trainType='discRegAdam')

Discretising the prediction into the loss function had no effect, further research will need to be conducted if there are loss fucntions fit for this task
For now, the definition of the normal regression adam trainer will be trained to give this definition of accuracy, as it's a different metric than loss.